# Task 1

In [16]:
import yfinance as yf

## This was just a preliminary step to see the start date of the selected ETF's and then
## decide which to use. We wanted to make sure they were at least 10 years old to get the
## full data. 

## This was the ChatGPT prompt used for this subtask: Write a Python script using the yfinance 
## library to find the earliest trading date for a list of ETFs. Then I substitued the list
## of ETFs

etfs = ['XLB', 'VOX', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'VNQ', 'XLU', 'XLV', 'XLY']
etf_start_dates = {}

for etf in etfs:
    ticker = yf.Ticker(etf)
    hist_data = ticker.history(period="max")
    first_date = hist_data.index.min().strftime('%Y-%m-%d') 
    etf_start_dates[etf] = first_date

print(etf_start_dates)


{'XLB': '1998-12-22', 'VOX': '2004-09-29', 'XLE': '1998-12-22', 'XLF': '1998-12-22', 'XLI': '1998-12-22', 'XLK': '1998-12-22', 'XLP': '1998-12-22', 'VNQ': '2004-09-29', 'XLU': '1998-12-22', 'XLV': '1998-12-22', 'XLY': '1998-12-22'}


In [97]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

#etfs = ['XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLRE', 'XLU', 'XLV', 'XLY']
etfs = ['VNQ', 'VOX','XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY']

## This downloads the necessary data with adjusted close and changes to returns.
## We originally ran into issue with pulling the most recent month that had incomplete data,
## so that explains the 121 to 120 month conversion. 

data = yf.download(etfs, period='max', interval="1mo")
adj_close = data['Adj Close']
monthly_returns = adj_close.pct_change().dropna()
recent_121_months = monthly_returns.tail(121)
recent_120_months = recent_121_months.drop(recent_121_months.index[-1])
print(recent_120_months.head())
recent_120_months.to_csv("etf_vanguard_recent_120_months.csv")


[*********************100%***********************]  11 of 11 completed

Ticker                          VNQ       VOX       XLB       XLE       XLF  \
Date                                                                          
2014-10-01 00:00:00+00:00  0.109593  0.020848 -0.019471 -0.030348  0.033169   
2014-11-01 00:00:00+00:00  0.020003  0.008865  0.015703 -0.086937  0.023490   
2014-12-01 00:00:00+00:00  0.005337 -0.057724 -0.011798 -0.008269  0.013525   
2015-01-01 00:00:00+00:00  0.083082 -0.019004 -0.012132 -0.039675 -0.064798   
2015-02-01 00:00:00+00:00 -0.036742  0.066177  0.079681  0.045929  0.058235   

Ticker                          XLI       XLK       XLP       XLU       XLV  \
Date                                                                          
2014-10-01 00:00:00+00:00  0.043658  0.020414  0.042027  0.090041  0.056326   
2014-11-01 00:00:00+00:00  0.030791  0.048101  0.055449  0.011656  0.034785   
2014-12-01 00:00:00+00:00 -0.005799 -0.026830 -0.016430  0.026522 -0.017670   
2015-01-01 00:00:00+00:00 -0.030109 -0.030047 -0.00

In [99]:
## This finds the necessary statistics (expected return and risk) and also checks
## demeaned returns.

mean_returns = recent_120_months.mean()
std_dev = recent_120_months.std()

print("Expected Monthly Returns:\n", mean_returns)
print("Standard Deviations (Risk):\n", std_dev)

demeaned_returns = recent_120_months.subtract(mean_returns, axis=1)
#print(demeaned_returns.head())
demeaned_returns.to_csv("demeaned_etf_vanguard_recent_120_months.csv")




Expected Monthly Returns:
 Ticker
VNQ    0.007356
VOX    0.007109
XLB    0.008833
XLE    0.006898
XLF    0.010635
XLI    0.010845
XLK    0.017181
XLP    0.008028
XLU    0.009257
XLV    0.009565
XLY    0.011823
dtype: float64
Standard Deviations (Risk):
 Ticker
VNQ    0.053606
VOX    0.052237
XLB    0.056449
XLE    0.087967
XLF    0.057529
XLI    0.053936
XLK    0.055710
XLP    0.037494
XLU    0.045697
XLV    0.041404
XLY    0.057718
dtype: float64


# Task 2

In [90]:
import numpy as np
import pandas as pd

## Here, we assume the monthly returns are already calculated in 'monthly_returns' from Task 1, and we use
## the client's original portfolio allocations from the project document. The allocations, total value, and 
## weights below were done with the following prompt:
## Create a Python script that defines a dictionary based on these ETF allocations. Then, calculate the total value 
## of all allocations, and then create another dictionary that holds the weight of each ETF as a proportion of the total value.

allocations = {
    'VNQ': 6750000,  # Real Estate
    'VOX': 4950000,  # Communication Services
    'XLB': 6300000,  # Materials
    'XLE': 2250000,  # Energy
    'XLF': 6300000,  # Financials
    'XLI': 4050000,  # Industrials
    'XLK': 5400000,  # Technology
    'XLP': 2250000,  # Consumer Staples
    'XLU': 1350000,  # Utilities
    'XLV': 1800000,  # Health Care
    'XLY': 3600000   # Consumer Discretionary
}


total_value = sum(allocations.values())
weights = {etf: value / total_value for etf, value in allocations.items()}
weights_vector = np.array(list(weights.values())) 

expected_returns = mean_returns  
cov_matrix = recent_120_months.cov()  
portfolio_expected_return = np.dot(weights_vector, expected_returns) ## weighted average 

portfolio_variance = np.dot(weights_vector.T, np.dot(cov_matrix, weights_vector)) ##sigma squared p = w' * covmat * w
portfolio_std_dev = np.sqrt(portfolio_variance)

## Here we are assuming a risk-free rate of 0.03 (3%) APR as stated in class by Prof. Sury, then
## adjusting to monthly risk-free rate
risk_free_rate = 0.03 / 12
sharpe_ratio = (portfolio_expected_return - risk_free_rate) / portfolio_std_dev


## For the printing of the metrics, I asked ChatGPT: 
## Can you use the portfolio expected return, standard deviation, and sharpe ratio that I calculated
## and print them in a summary format. Then can you do the same with the assets, the weights, and the
## expected return so the metrics print for each individual asset.

## I then simply altered this format for the metric printing in the rest of the code.

print("Client's Original Portfolio Metrics:")
print(f"Expected Monthly Return: {portfolio_expected_return:.4f}")
print(f"Portfolio Standard Deviation (Risk): {portfolio_std_dev:.4f}")
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")

portfolio_metrics = pd.DataFrame({
    'ETF': list(weights.keys()),
    'Weight': list(weights.values()),
    'Expected Return': expected_returns.values,
})

portfolio_summary = pd.Series({
    'Expected Return': portfolio_expected_return,
    'Standard Deviation': portfolio_std_dev,
    'Sharpe Ratio': sharpe_ratio
})

print(portfolio_metrics)
print("\nPortfolio Summary:")
print(portfolio_summary)



Client's Original Portfolio Metrics:
Expected Monthly Return: 0.0100
Portfolio Standard Deviation (Risk): 0.0464
Sharpe Ratio: 0.1616
    ETF  Weight  Expected Return
0   VNQ    0.15         0.007356
1   VOX    0.11         0.007109
2   XLB    0.14         0.008833
3   XLE    0.05         0.006898
4   XLF    0.14         0.010635
5   XLI    0.09         0.010845
6   XLK    0.12         0.017181
7   XLP    0.05         0.008028
8   XLU    0.03         0.009257
9   XLV    0.04         0.009565
10  XLY    0.08         0.011823

Portfolio Summary:
Expected Return       0.010001
Standard Deviation    0.046425
Sharpe Ratio          0.161574
dtype: float64


# Task 8

### Task 8 compiled into one file to run all steps

In [5]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from scipy.optimize import minimize
import numpy as np

print("Welcome to the Soup Kitchen Saturdays Portfolio Optimizer!")
securities = input("Please enter the tickers for the securities in your portfolio, separated by commas: ").split(',')
securities = [s.strip().upper() for s in securities]


print("Downloading historical data for the specified securities...")
data = yf.download(securities, period='max', interval="1mo")


adj_close = data['Adj Close']
monthly_returns = adj_close.pct_change().dropna()
recent_121_months = monthly_returns.tail(121)
recent_120_months = recent_121_months.drop(recent_121_months.index[-1])


weights = {}
total_weight = 0
tolerance = 1e-6 
while abs(total_weight - 1) > tolerance:
    print("Enter the original weights for each security in decimal (i.e. They should sum to 1)")
    weights = {}  
    total_weight = 0 

    for security in securities:
        weight = float(input(f"Enter the weight for {security}: "))
        weights[security] = weight
        total_weight += weight
    
    if abs(total_weight - 1) > tolerance:
        print(f"The weights you entered sum to {total_weight:.4f}, but they must sum to 1. Please try again.\n")

print(f"Weights: {weights}")


risk_free_rate = float(input("Enter the risk-free rate in APR (as a decimal, e.g., 0.03 for 3%): "))
risk_free_rate = risk_free_rate/12


lower_bound_input = input("Enter the lower bound for the weights (e.g., 0 for no shorting, 'None' for no constraint): ")
upper_bound_input = input("Enter the upper bound for the weights (e.g., 1 for 100%, 'None' for no constraint): ")


if lower_bound_input.lower() == 'none':
    lower_bound = None  
else:
    lower_bound = float(lower_bound_input)

if upper_bound_input.lower() == 'none':
    upper_bound = None  
else:
    upper_bound = float(upper_bound_input)


print("\nCollected Data:")
print(f"Securities: {securities}")
print(f"Weights: {weights}")
print(f"Monthly risk-free rate: {risk_free_rate}")
print(f"Weight bounds: Lower={lower_bound}, Upper={upper_bound}")
recent_120_months.to_csv("task8_120_months_returns.csv")


## ORIGINAL PORTFOLIO METRICS (FROM TASK 1 AND 2)
covmat = recent_120_months.cov()
orig_weight_vector = np.array(list(weights.values()))
orig_portfolio_expected_return = np.dot(orig_weight_vector, recent_120_months.mean())
orig_portfolio_variance = np.dot(orig_weight_vector.T, np.dot(covmat, orig_weight_vector)) 
orig_portfolio_std_dev = np.sqrt(orig_portfolio_variance)
orig_portfolio_sharpe_ratio = (orig_portfolio_expected_return - risk_free_rate)/orig_portfolio_std_dev

orig_port_weights = pd.Series(weights, index=recent_120_months.columns)
print("\nClient's Original Portfolio Metrics:")
print(f"Expected Monthly Return: {orig_portfolio_expected_return:.4f}")
print(f"Portfolio Standard Deviation (Risk): {orig_portfolio_std_dev:.4f}")
print(f"Sharpe Ratio: {orig_portfolio_sharpe_ratio:.4f}")


## GMV WITH Ax = b CALCULATIONS

muvec = recent_120_months.mean()
variances = recent_120_months.var()
covmat = recent_120_months.cov()

inv_covmat = np.linalg.inv(covmat)
covmat_two = 2 * covmat

n_assets = len(recent_120_months.columns) 
ones_vec = np.ones(n_assets)

A_top = np.hstack([covmat_two.values, ones_vec.reshape(-1, 1)])
A_bottom = np.hstack([ones_vec.reshape(1, -1), np.zeros((1, 1))])
A_matrix = np.vstack([A_top, A_bottom])

b_vector = np.hstack([np.zeros(n_assets), 1])
inv_A_matrix = np.linalg.inv(A_matrix)
x_solution = np.matmul(inv_A_matrix, b_vector)

gmv_weights = x_solution[:-1]

gmv_portfolio_weights = pd.Series(gmv_weights, index=recent_120_months.columns)
print("\nGMV Portfolio Weights:")
print(gmv_portfolio_weights)
gmv_expected_return = np.dot(gmv_weights, muvec)
gmv_std_dev = np.sqrt(np.dot(gmv_weights.T, np.dot(covmat, gmv_weights)))
gmv_sharpe_ratio = (gmv_expected_return - risk_free_rate) / gmv_std_dev

print("\nGMV Portfolio Metrics:")
print(f"Expected Return: {gmv_expected_return:.4f}")
print(f"Standard Deviation (Risk): {gmv_std_dev:.4f}")
print(f"Sharpe Ratio: {gmv_sharpe_ratio:.4f}")



## GMV MATRIX-ANALYTIC SOLUTION

numerator = np.dot(inv_covmat,ones_vec)
denominator = np.dot(ones_vec.T, np.dot(inv_covmat, ones_vec))

gmv_weights_2 = numerator / denominator
gmv_portfolio_weights = pd.Series(gmv_weights_2, index=recent_120_months.columns)
print("\nAnalytic GMV Portfolio Weights:")
print(gmv_portfolio_weights)

gmv_expected_return_2 = np.dot(gmv_weights_2, muvec)
gmv_std_dev_2 = np.sqrt(np.dot(gmv_weights_2.T, np.dot(covmat, gmv_weights_2)))
gmv_sharpe_ratio_2 = (gmv_expected_return_2 - risk_free_rate) / gmv_std_dev_2

print("\nAnalytic GMV Portfolio Metrics:")
print(f"Expected Return: {gmv_expected_return_2:.4f}")
print(f"Standard Deviation (Risk): {gmv_std_dev_2:.4f}")
print(f"Sharpe Ratio: {gmv_sharpe_ratio_2:.4f}")


## EFFICIENT PORTFOLIO CALCULATIONS

M = np.column_stack((muvec, ones_vec))
B = np.dot(np.dot(M.T, inv_covmat), M)

w_orig = np.array(list(weights.values())) 
w_transpose_mu = np.dot(w_orig, muvec) 

mu_star = np.array([w_transpose_mu, 1])

inv_B = np.linalg.inv(B) 
w_ep = np.dot(np.dot(inv_covmat, M), np.dot(inv_B, mu_star))

ep_weights = pd.Series(w_ep, index=muvec.index)
print("\nEfficient Portfolio Weights:")
print(ep_weights)
ep_expected_return = np.dot(ep_weights, muvec)
ep_std_dev = np.sqrt(np.dot(ep_weights.T, np.dot(covmat, ep_weights)))
ep_sharpe_ratio = (ep_expected_return - risk_free_rate) / ep_std_dev

print(f"\nEfficient Portfolio Expected Return: {ep_expected_return:.4f}")
print(f"Efficient Portfolio Standard Deviation (Risk): {ep_std_dev:.4f}")
print(f"Efficient Portfolio Sharpe Ratio: {ep_sharpe_ratio:.4f}")


## OPTIMAL RISKY PORTFOLIO NO CONSTRAINTS CALCULATIONS

excess_returns = muvec - risk_free_rate * ones_vec
orp_numerator = np.dot(inv_covmat, excess_returns)
orp_denominator = np.dot(ones_vec.T, orp_numerator)

w_orp = orp_numerator / orp_denominator
orp_weights = pd.Series(w_orp, index=muvec.index)
print("\nOptimal Risky Portfolio Weights:")
print(orp_weights)

orp_expected_return = np.dot(orp_weights, muvec)
orp_std_dev = np.sqrt(np.dot(orp_weights.T, np.dot(covmat, orp_weights)))
orp_sharpe_ratio = (orp_expected_return - risk_free_rate) / orp_std_dev

print(f"\nOptimal Risky Portfolio Expected Return: {orp_expected_return:.4f}")
print(f"Optimal Risky Portfolio Standard Deviation (Risk): {orp_std_dev:.4f}")
print(f"Optimal Risky Portfolio Sharpe Ratio: {orp_sharpe_ratio:.4f}")



## OPTIMAL RISKY PORTFOLIO WITH CONSTRAINTS CALCULATIONS

def objective(weights):
    
    con_portfolio_return = np.dot(weights.T, muvec) - risk_free_rate
    con_portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(covmat, weights)))
    con_sharpe_ratio = con_portfolio_return / con_portfolio_stddev
    return -con_sharpe_ratio


n = len(muvec)
bounds = [(lower_bound, upper_bound) for _ in range(n)]
constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}


initial_guess = np.array([1/n] * n)
result = minimize(objective, initial_guess, bounds=bounds, constraints=constraints, method='SLSQP')

if result.success:
    con_optimized_weights = result.x
    con_orp_weights = pd.Series(con_optimized_weights, index=muvec.index)
    print("\nConstrained Optimal Risky Portfolio Weights:")
    print(con_orp_weights)

    
    con_orp_expected_return = np.dot(con_orp_weights, muvec)
    con_orp_std_dev = np.sqrt(np.dot(con_orp_weights.T, np.dot(covmat, con_orp_weights)))
    con_orp_sharpe_ratio = (con_orp_expected_return - risk_free_rate) / con_orp_std_dev

    
    print(f"\nConstrained ORP Expected Return: {con_orp_expected_return:.4f}")
    print(f"Constrained ORP Standard Deviation (Risk): {con_orp_std_dev:.4f}")
    print(f"Constrained ORP Sharpe Ratio: {con_orp_sharpe_ratio:.4f}")
else:
    print("Optimization failed:", result.message)



## FORMATTING AND WRITING TO EXCEL

portfolios = ['ORIGPORT', 'GMVPORT', 'EFPORT', 'ORP', 'Constrained ORP']
columns = securities + ['TOTAL', 'Expected Return', 'Standard Deviation', 'Sharpe Ratio']
## For this step, I submitted the printing code I had and asked ChatGPT: 
##  I just would like you to alter this code so it displays the weights in percentage out to two decimals
## and includes a percent sign with the format.

weights_data = {
    'ORIGPORT': [f"{round(w * 100, 2)}%" for w in orig_port_weights.values] + [f"{round(sum(orig_port_weights.values) * 100, 2)}%"],
    'GMVPORT': [f"{round(w * 100, 2)}%" for w in gmv_portfolio_weights.values] + [f"{round(sum(gmv_portfolio_weights) * 100, 2)}%"],
    'EFPORT': [f"{round(w * 100, 2)}%" for w in ep_weights.values] + [f"{round(sum(ep_weights) * 100, 2)}%"],
    'ORP': [f"{round(w * 100, 2)}%" for w in orp_weights.values] + [f"{round(sum(orp_weights) * 100, 2)}%"],
    'Constrained ORP': [f"{round(w * 100, 2)}%" for w in con_orp_weights.values] + [f"{round(sum(con_orp_weights) * 100, 2)}%"]
}

metrics_data = {
    'ORIGPORT': [orig_portfolio_expected_return, orig_portfolio_std_dev, orig_portfolio_sharpe_ratio], 
    'GMVPORT': [gmv_expected_return, gmv_std_dev, gmv_sharpe_ratio],
    'EFPORT': [ep_expected_return, ep_std_dev, ep_sharpe_ratio],
    'ORP': [orp_expected_return, orp_std_dev, orp_sharpe_ratio],
    'Constrained ORP': [con_orp_expected_return, con_orp_std_dev, con_orp_sharpe_ratio]
}


data = []
for portfolio in portfolios:
    weights = weights_data[portfolio]  
    metrics = metrics_data[portfolio]  
    data.append(weights + metrics)     

df = pd.DataFrame(data, index=portfolios, columns=columns)


df.to_excel("Portfolio_Report_General.xlsx", index=True)

print("Portfolio report has been written to Portfolio_Report_General.xlsx")


Welcome to the Soup Kitchen Saturdays Portfolio Optimizer!


[*********************100%***********************]  11 of 11 completed


Enter the original weights for each security in decimal (i.e. They should sum to 1)
Weights: {'VNQ': 0.15, 'VOX': 0.11, 'XLB': 0.14, 'XLE': 0.05, 'XLF': 0.14, 'XLI': 0.09, 'XLK': 0.12, 'XLP': 0.05, 'XLU': 0.03, 'XLV': 0.04, 'XLY': 0.08}

Collected Data:
Securities: ['VNQ', 'VOX', 'XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY']
Weights: {'VNQ': 0.15, 'VOX': 0.11, 'XLB': 0.14, 'XLE': 0.05, 'XLF': 0.14, 'XLI': 0.09, 'XLK': 0.12, 'XLP': 0.05, 'XLU': 0.03, 'XLV': 0.04, 'XLY': 0.08}
Monthly risk-free rate: 0.0025
Weight bounds: Lower=0.0, Upper=1.0

Client's Original Portfolio Metrics:
Expected Monthly Return: 0.0100
Portfolio Standard Deviation (Risk): 0.0464
Sharpe Ratio: 0.1616

GMV Portfolio Weights:
Ticker
VNQ   -0.276204
VOX    0.137662
XLB   -0.124850
XLE    0.008203
XLF    0.170162
XLI   -0.277683
XLK   -0.013431
XLP    0.497922
XLU    0.327538
XLV    0.418121
XLY    0.132560
dtype: float64

GMV Portfolio Metrics:
Expected Return: 0.0091
Standard Deviation (Risk): 0.03